Cambios: 
    Optimizacion: 
        Uso de EstimatorV2 de qiskit_aer.primitive con backend_options especificos. Uso de Session y Estimator de qiskit_ibm_runtime para hardware real.
        Backend configuration + load/save with pickle
        Uso de metodos de gradientes de qiskit_algorithms.gradients.
        Tipo de dato torch.float32 para tensores.


## Introduction

The Quantum Generative Adversarial Network (QGAN) [[1]](https://github.com/Qiskit/textbook/blob/main/notebooks/quantum-machine-learning/qgan.ipynb)  [[2]](https://arxiv.org/abs/1406.2661) we propose consists of two Quantum Neural Network (QNN) [[3]](https://qiskit-community.github.io/qiskit-machine-learning/tutorials/01_neural_networks.html): a generator and a discriminator. The generator is responsible for creating synthetic data samples. The discriminator evaluates the authenticity of the created samples by distinguishing between real and generated data. Through an adversarial training process, both networks continuously improve, leading to the generation of increasingly realistic data. 
This fully quantum approach benefits from the strengths of quantum state preparation and gradient calculation combined with classical optimizators [[4]](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
The data used to train the QGAN in this implementation is a probability distributions.

This implementation uses aer_simulator_statevector.

## Implementation (statevector simulation)

In [10]:
#--- INSTALATION INSTRUCTIONS ---#

# For linux 64-bit systems,
#uname -a

# Conda quick installation
#mkdir -p ~/miniconda3
#wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
#bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
#rm ~/miniconda3/miniconda.sh

# Create enviroment with conda
#conda create -n myenv python=3.10
#conda activate myenv
#pip install qiskit==1.4.5 qiskit-machine-learning==0.8.4 'qiskit-machine-learning[sparse]' qiskit_aer qiskit_algorithms torch matplotlib pylatexenc ipykernel
# IMPORTANT: Make sure you are on 3.10
# May need to restart the kernel after instalation

#--- Imports ---#
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import random_statevector, Statevector, SparsePauliOp
from qiskit.circuit.library import RealAmplitudes, EfficientSU2
from qiskit.primitives import StatevectorEstimator
from qiskit.visualization import plot_histogram
from qiskit import qpy
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_machine_learning.neural_networks import EstimatorQNN # Downgrade to qiskit 1.x so is compatible with qiskit-machine-learning 0.8.2
from qiskit_machine_learning.gradients import ParamShiftEstimatorGradient, SPSAEstimatorGradient

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_aer.backends.backendconfiguration import AerBackendConfiguration
from qiskit_aer.backends.backendproperties import AerBackendProperties
from qiskit_aer.primitives import EstimatorV2 as EstimatorV2_sim

from qiskit_ibm_runtime import EstimatorV2 as EstimatorV2_rh, QiskitRuntimeService, Session
from qiskit_ibm_runtime.options import EstimatorOptions

from qiskit_algorithms.gradients import ReverseEstimatorGradient

import numpy as np
import torch
import matplotlib.pyplot as plt
import time
import os
import signal
import datetime as dt
import pickle

In [11]:
#- Configuration -#

# Training configuration dict
train_config = {
    'execution_type': "noisy_simulation", #noiseless_simulation
    'n_qubits': 4,
    'seed': 2,
    'id': None, # For different circuits or training parameters
    'reset_data': False,

    'create_circuits': False, # Create circuits manually or load from file
    'gradient_method': "REG", # qiskit_algorithms.gradients For now: PSR, SPSA and REG
    'max_iterations': 1000,
    'gen_iterations': 1,
    'disc_iterations': 1,
    'save_loss_iterations': 10, # Calculate extra forward pass to save loss
    'print_progress_iterations': 10,

    'training_data_file': None, # Automatically created with manage_files function
    'circuits_file': None, # Automatically created with manage_files function
    'backend_file': None # Automatically created with manage_files function
}

# File management
def manage_files(data_folder_name = 'data', implementation_name = 'fullyq_torch', execution_type_name = train_config['execution_type'], training_data_file_name = 'training_data', circuits_file_name = 'circuits', backend_file_name = 'backend'):
    data_folder = data_folder_name + '/' + implementation_name + '/' + execution_type_name + '/' + 'q' + str(train_config['n_qubits']) + '/' + 'seed' + str(train_config['seed']) + '/'
    if train_config['id'] is not None:
        data_folder = data_folder + '/' + str(train_config['id']) + '/' 
    training_data_file = data_folder + training_data_file_name + '.pth'
    circuits_file = data_folder + circuits_file_name + '.qpy'
    backend_file = data_folder + backend_file_name + '.pkl'

    # Create folders if they do not exist
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

    return training_data_file, circuits_file, backend_file

if ((train_config['training_data_file'] is None) and (train_config['circuits_file'] is None) and (train_config['backend_file'] is None)):
    train_config['training_data_file'], train_config['circuits_file'], train_config['backend_file'] = manage_files()

In [12]:
#- Backend configuration -#
backend_config = {
    # Real backend
    'name': "ibm_basquecountry",
    'channel': "ibm_quantum_platform",

    # Noisy backend
    'reset_backend': False, # Get current backend state or load from file
    'timestamp': dt.datetime(year=2025, month=12, day=5, hour = 10, tzinfo=dt.timezone.utc), # Get exact backend state, None to get current state (no he conseguido q funcione)

    # Noiseless backend
    'sim_options': {
        'method': 'statevector',
        #'device': 'GPU',
        'precision': 'single',       # Significant speedup 
        #'cuStateVec_enable': True,   # NVIDIA library optimization [9]
        #'batched_shots_gpu': True,   # Parallelize batch on GPU [9]
        #'blocking_enable': False,     # Disable chunking; simulation fits in VRAM 
        #'seed_simulator': train_config['seed']
    }
}

# # Save account
# QiskitRuntimeService.save_account(
#     token="",
#     instance="crn:v1:bluemix:public:quantum-computing:eu-de:a/cb804b30dfcb48b890393bfd6e41e9c2:4cb40c64-a531-4c13-b39c-e04c31185259::",
#     set_as_default = True,
#     overwrite=True
# )

def reset_backend():
    service = QiskitRuntimeService(channel=backend_config['channel'])
    real_backend = service.backend(backend_config['name']) #backend = service.least_busy(min_num_qubits=30)

    backend = AerSimulator.from_backend(real_backend, **backend_config['sim_options']) # Get current backend state
    #backend.set_options(seed_simulator=train_config['seed']) RANDOM STATE? TODO
    #backend.set_options(**backend_config['sim_options'])

    backend_dict = {
        'timestamp': dt.datetime.now(dt.timezone.utc),
        'configuration': real_backend.configuration().to_dict(),
        'options': backend.options,
        'properties': real_backend.properties().to_dict(),
        'target': real_backend.target_history(),
        'noise_model': NoiseModel.from_backend(real_backend),
    }

    # # Example for simulator backend
    # backend_dict = {
    #     'timestamp': dt.datetime.now(dt.timezone.utc),
    #     'configuration': backend.configuration().to_dict(),
    #     'options': backend.options, # Options object saved directly
    #     'properties': backend.properties().to_dict(),
    #     'target': backend.target, # Target object saved directly
    #     'noise_model': NoiseModel.from_backend(backend),
    #     'sim_options': {'shots': 1024}
    #
    #     # With timestamp
    #      'target_h': real_backend.target_history(datetime=backend_config['timestamp']),
    #      'noise_model_h': NoiseModel.from_backend_properties(real_backend.properties(datetime=backend_config['timestamp'])), # No functiona: error por falta de datos en properties
    # }

    # Save to a single .pkl file
    with open(train_config['backend_file'], "wb") as f:
        pickle.dump(backend_dict, f)



if train_config['execution_type'] == "real_hardware":
    service = QiskitRuntimeService(channel=backend_config['channel']) # Execution in real hardware
    backend = service.backend(backend_config['name']) #backend = service.least_busy(min_num_qubits=30)

    pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

    session = Session(backend=backend)
    precision = 0.015625
    estimator = EstimatorV2_rh(mode=session, options=EstimatorOptions(precision=precision))

elif train_config['execution_type'] == "noisy_simulation":
    # Load backend configuration
    try:
        with open(train_config['backend_file'], "rb") as f:
            backend_dict = pickle.load(f)
    except FileNotFoundError:
        print("Backend data file not found. Resetting backend configuration.")
        reset_backend()
        with open(train_config['backend_file'], "rb") as f:
            backend_dict = pickle.load(f)

    backend = AerSimulator(
        configuration=AerBackendConfiguration.from_dict(backend_dict['configuration']),
        properties=AerBackendProperties.from_dict(backend_dict['properties']),
        target = backend_dict['target'],
        **backend_dict['options']
    )

    pm = generate_preset_pass_manager(optimization_level=3, backend=backend)

    precision = 0.015625
    estimator = EstimatorV2_sim(
        options = {
            "default_precision": precision,
            #'seed_estimator': train_config['seed'],
            "backend_options": backend_config['sim_options'],
            "run_options": {} #TODO configurar mejor?
        })

else:
    backend = AerSimulator(**backend_config['sim_options'])

    precision = 0.0
    estimator = EstimatorV2_sim(
        options = {
            "default_precision": precision,
            #'seed_estimator': train_config['seed'],
            "backend_options": backend_config['sim_options'],
            "run_options": {} #TODO configurar mejor?
        })

    #pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
    pm = None


# # Select device torch? Yata en sim_options no? TODO

Backend data file not found. Resetting backend configuration.


In [13]:
#- Create quantum circuits -#

# Create real data sample circuit
def generate_real_circuit():
    n_qubits = train_config['n_qubits']

    # sv = random_statevector(2**N_QUBITS, seed=SEED)
    # qc = QuantumCircuit(N_QUBITS)
    # qc.prepare_state(sv, qc.qubits, normalize=True)

    qc = QuantumCircuit(n_qubits)
    qc.h(range(n_qubits-1))
    qc.cx(n_qubits-2, n_qubits-1)
    return qc


# Create generator
def generate_generator():
    n_qubits = train_config['n_qubits']

    qc = RealAmplitudes(n_qubits,
                        reps=3, # Number of layers
                        parameter_prefix='θ_g',
                        name='Generator')
    
    return qc.decompose()


# Create discriminator
def generate_discriminator():
    n_qubits = train_config['n_qubits']

    qc = EfficientSU2(n_qubits,
                      entanglement="reverse_linear",
                      reps=1, # Number of layers
                      parameter_prefix='θ_d',
                      name='Discriminator').decompose()


    param_index = qc.num_parameters

    for i in reversed(range(n_qubits - 1)):
        qc.cx(i, n_qubits - 1)

    #qc.rx(disc_weights[param_index], N_QUBITS-1); param_index += 1
    qc.ry(Parameter("θ_d["+str(param_index)+"]"), n_qubits-1); param_index += 1
    qc.rz(Parameter("θ_d["+str(param_index)+"]"), n_qubits-1); param_index += 1
    
    return qc


# Create quantum circuits
def create_circuits():
    real_circuit = generate_real_circuit()
    generator_circuit = generate_generator()
    discriminator_circuit = generate_discriminator()

    with open(train_config['circuits_file'], 'wb') as fd:
        qpy.dump([real_circuit, generator_circuit, discriminator_circuit], fd)

# Load circuits
if train_config['create_circuits']:
    create_circuits()

try:
    with open(train_config['circuits_file'], 'rb') as fd:
        circuits = qpy.load(fd)
except FileNotFoundError:
    print("Circuits file not found. Creating new circuits file.")
    create_circuits()
    with open(train_config['circuits_file'], 'rb') as fd:
        circuits = qpy.load(fd)
    
real_circuit = circuits[0]
generator_circuit = circuits[1]
discriminator_circuit = circuits[2]

Circuits file not found. Creating new circuits file.


In [14]:
#- Set up training quantum circuits -#
def generate_training_circuits(real_circuit, generator_circuit, discriminator_circuit):
    n_qubits = train_config['n_qubits']

    # Connect real data and discriminator
    real_disc_circuit = QuantumCircuit(n_qubits)
    real_disc_circuit.compose(real_circuit, inplace=True)
    real_disc_circuit.compose(discriminator_circuit, inplace=True)

    # Connect generator and discriminator
    gen_disc_circuit = QuantumCircuit(n_qubits)
    gen_disc_circuit.compose(generator_circuit, inplace=True)
    gen_disc_circuit.compose(discriminator_circuit, inplace=True)

    # Gradient computation method
    if train_config['gradient_method'] == 'SPSA':
        gradient = SPSAEstimatorGradient(estimator=estimator)
    elif train_config['gradient_method'] == 'REG':
        gradient = ReverseEstimatorGradient()
    else:
        gradient = ParamShiftEstimatorGradient(estimator=estimator)

    # Observables
    H1 = SparsePauliOp.from_list([("Z" + "I"*(n_qubits-1), 1.0)])
    N_DPARAMS = discriminator_circuit.num_parameters

    # specify QNN to update generator parameters
    gen_qnn = EstimatorQNN(circuit=gen_disc_circuit,
                        input_params=gen_disc_circuit.parameters[:N_DPARAMS], # fixed parameters (discriminator parameters)
                        weight_params=gen_disc_circuit.parameters[N_DPARAMS:], # parameters to update (generator parameters)
                        estimator=estimator,
                        observables=[H1],
                        gradient=gradient,
                        default_precision=precision,
                        pass_manager=pm
                        )

    # specify QNN to update discriminator parameters regarding to fake data
    disc_fake_qnn = EstimatorQNN(circuit=gen_disc_circuit,
                            input_params=gen_disc_circuit.parameters[N_DPARAMS:], # fixed parameters (generator parameters)
                            weight_params=gen_disc_circuit.parameters[:N_DPARAMS], # parameters to update (discriminator parameters)
                            estimator=estimator,
                            observables=[H1],
                            gradient=gradient,
                            default_precision=precision,
                            pass_manager=pm
                            )

    # specify QNN to update discriminator parameters regarding to real data
    disc_real_qnn = EstimatorQNN(circuit=real_disc_circuit,
                            input_params=[], # no input parameters
                            weight_params=gen_disc_circuit.parameters[:N_DPARAMS], # parameters to update (discriminator parameters)
                            estimator=estimator,
                            observables=[H1],
                            gradient=gradient,
                            default_precision=precision,
                            pass_manager=pm
                            )
    
    return gen_qnn, disc_fake_qnn, disc_real_qnn

gen_qnn, disc_fake_qnn, disc_real_qnn = generate_training_circuits(real_circuit, generator_circuit, discriminator_circuit)

In [15]:
#- Restore parameters and model states -#

# Reset all data training
def reset_data(n_gen_params, n_disc_params):
    np.random.seed(train_config['seed'])

    init_gen_params = np.random.uniform(low=-np.pi, high=np.pi, size=(n_gen_params,))
    init_disc_params = np.random.uniform(low=-np.pi, high=np.pi, size=(n_disc_params,))

    gen_params = torch.tensor(init_gen_params, requires_grad=True, dtype = torch.float32)
    disc_params = torch.tensor(init_disc_params, requires_grad=True, dtype = torch.float32)

    optimizer_g = torch.optim.Adam([gen_params], lr=0.005)
    optimizer_d = torch.optim.Adam([disc_params], lr=0.005)

    torch.save({
        'init_gen_params': init_gen_params,
        'init_disc_params': init_disc_params,
        'gen_params': gen_params,
        'disc_params': disc_params,
        'best_gen_params': init_gen_params,
        'optimizer_g_state': optimizer_g.state_dict(),
        'optimizer_d_state': optimizer_d.state_dict(),
        'current_epoch': 0,
        "metrics": {
            "gloss": {},
            "dloss": {},
            "kl_div": {},
        },
        'random_state': np.random.get_state()
    }, train_config['training_data_file'])


# Load parameters and training states
if train_config['reset_data']:
    reset_data(generator_circuit.num_parameters, discriminator_circuit.num_parameters)

try:
    params = torch.load(train_config['training_data_file'], weights_only=False)
except FileNotFoundError:
    print("Training data file not found. Resetting parameters.")
    reset_data(generator_circuit.num_parameters, discriminator_circuit.num_parameters)
    params = torch.load(train_config['training_data_file'], weights_only=False)

np.random.set_state(params['random_state'])

gen_params = params['gen_params']
disc_params = params['disc_params']

optimizer_g = torch.optim.Adam([gen_params])
optimizer_d = torch.optim.Adam([disc_params])

optimizer_g.load_state_dict(params['optimizer_g_state'])
optimizer_d.load_state_dict(params['optimizer_d_state'])

current_epoch = params['current_epoch']
gloss = params['metrics']['gloss']
gen_loss = list(gloss)[-1] if (gloss) else None
dloss = params['metrics']['dloss']
disc_loss = list(dloss)[-1] if (dloss) else None
kl_div = params['metrics']['kl_div']
min_kl_div = np.min(list(kl_div.values())) if (kl_div) else float('inf')
best_gen_params = params['best_gen_params']

Training data file not found. Resetting parameters.


In [16]:
#- Manage training interruption -#

# Class to manage training interruption
class Interrupter:
    def __init__(self):
        self.kill_now = False
        self.interrupt_count = 0

        # Intercept the Ctrl+C signal
        signal.signal(signal.SIGINT, self.handle_signal)
        # Intercept the termination signal (useful for Docker/systems)
        #signal.signal(signal.SIGTERM, self.handle_signal)

    def handle_signal(self, signum, frame):
        self.interrupt_count += 1
        
        if self.interrupt_count == 1:
            # First Press: Enable graceful exit
            self.kill_now = True
            print("\nInterrupter: Termination signal received. The loop will stop after the current iteration. (Press Ctrl+C again to force quit)")
        
        elif self.interrupt_count >= 2:
            # Second Press: Force quit immediately
            print("\nInterrupter: [!] Force quit triggered! Terminating immediately.")
            # Restore default signal handler to avoid recursion
            signal.signal(signal.SIGINT, signal.SIG_DFL)
            # Raise the exception to stop execution right here
            raise KeyboardInterrupt

In [ ]:
disc_fake_qnn.forward(gen_params.detach(), disc_params.detach())[0,0] #TODO transpile circuits and observables before executing

ValueError: The number of qubits of the circuit (156) does not match the number of qubits of the ()-th observable (4).

In [ ]:
#- Training -#

D_STEPS = train_config['disc_iterations']
G_STEPS = train_config['gen_iterations']
C_STEPS = train_config['save_loss_iterations']

real_distribution_tensor = torch.from_numpy(Statevector(real_circuit).probabilities()) # Retrieve real data probability distribution 

interrupter = Interrupter()

if train_config['print_progress_iterations']:
    TABLE_HEADERS = "Epoch | Generator cost | Discriminator cost | KL Div. | Best KL Div. | Time |"
    print(TABLE_HEADERS)
start_time = time.time()

#--- Training loop ---#
try: # In case of interruption
    for epoch in range(current_epoch, train_config['max_iterations']+1):

        #--- Quantum discriminator parameter updates ---#
        for disc_train_step in range(D_STEPS):
            # Calculate discriminator cost
            if (disc_train_step == D_STEPS-1) and (epoch % C_STEPS == 0):
                value_dcost_fake = disc_fake_qnn.forward(gen_params.detach(), disc_params.detach())[0,0]
                value_dcost_real = disc_real_qnn.forward([], disc_params.detach())[0,0]
                disc_loss = ((value_dcost_real - value_dcost_fake)-2)/4
                dloss[epoch] = disc_loss

            # Caltulate discriminator gradient
            grad_dcost_fake = disc_fake_qnn.backward(gen_params.detach(), disc_params.detach())[1][0,0]
            grad_dcost_real = disc_real_qnn.backward([], disc_params.detach())[1][0,0]
            grad_dcost = grad_dcost_real - grad_dcost_fake
            grad_dcost = torch.tensor(grad_dcost, dtype = torch.float32)
            
            # Update discriminator parameters
            optimizer_d.zero_grad()
            disc_params.grad = grad_dcost.to(dtype=disc_params.dtype, device=disc_params.device)
            optimizer_d.step()

        #--- Quantum generator parameter updates ---#
        for gen_train_step in range(G_STEPS):
            # Calculate generator cost
            if (gen_train_step == G_STEPS-1) and (epoch % C_STEPS == 0):
                value_gcost = gen_qnn.forward(disc_params.detach(), gen_params.detach())[0,0]
                gen_loss = (value_gcost-1)/2
                gloss[epoch] = gen_loss

            # Calculate generator gradient
            grad_gcost = gen_qnn.backward(disc_params.detach(), gen_params.detach())[1][0,0]
            grad_gcost = torch.tensor(grad_gcost, dtype = torch.float32)

            # Update generator parameters
            optimizer_g.zero_grad()
            gen_params.grad = grad_gcost.to(dtype=gen_params.dtype, device=gen_params.device)
            optimizer_g.step()


        #--- Track KL and save best performing generator weights ---#
        gen_distribution_tensor = torch.from_numpy(Statevector(generator_circuit.assign_parameters(gen_params.detach().numpy())).probabilities()) # Retrieve probability distribution of generator with current parameters.

        # 3. Move to GPU (The speedup for large vectors is massive) # TODO
        # if torch.cuda.is_available():
        #     p = p.cuda()
        #     q = q.cuda()

        # Performance measurement function: uses Kullback Leibler Divergence to measures the distance between two distributions
        current_kl = torch.nn.functional.kl_div(input=gen_distribution_tensor.log(), target=real_distribution_tensor, reduction='sum').numpy() # reduction="batchnoseque" pa batches
        kl_div[epoch] = current_kl
        if min_kl_div > current_kl:
            min_kl_div = current_kl
            best_gen_params = gen_params.detach().numpy() # New best


        #--- Print progress ---#
        if train_config['print_progress_iterations'] and (epoch % train_config['print_progress_iterations'] == 0):
            for header, val in zip(TABLE_HEADERS.split('|'),
                                (epoch, gen_loss, disc_loss, current_kl, min_kl_div, (time.time() - start_time))):
                print(f"{val:.3g} ".rjust(len(header)), end="|")
            start_time = time.time()
            print()

        # In case of interruption
        if interrupter.kill_now:
            print("Interrupter: Graceful exit triggered. Breaking loop.")
            break
            
#--- Save parameters and optimizer states data ---#
finally:
    if train_config['execution_type'] == "real_hardware":
        session.close()
    
    torch.save({
        'init_gen_params': params['init_gen_params'],
        'init_disc_params': params['init_disc_params'],
        'best_gen_params': best_gen_params,
        'gen_params': gen_params,
        'disc_params': disc_params,
        'optimizer_g_state': optimizer_g.state_dict(),
        'optimizer_d_state': optimizer_d.state_dict(),
        'current_epoch': epoch+1,
        "metrics": {
            "gloss": gloss,
            "dloss": dloss,
            "kl_div": kl_div,
        },
        'random_state': np.random.get_state()
    }, train_config['training_data_file'])
    
    kl_div_data = list(kl_div.values())
    print("Training complete:", "\n   Data path:", train_config['training_data_file'], "\n   Best KLDiv:", np.min(kl_div_data), "in epoch", np.argmin(kl_div_data), "\n   Improvement:", kl_div_data[0]-np.min(kl_div_data))

Epoch | Generator cost | Discriminator cost | KL Div. | Best KL Div. | Time |


ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
import torch
import numpy as np
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit_algorithms.gradients import ReverseEstimatorGradient
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

# --- 3. QNN Definition ---
# The EstimatorQNN now uses the optimized backend and gradient method.
# Ensure 'input_params' and 'weight_params' are defined as per your circuit.
qnn_g = EstimatorQNN(
    circuit=generator_circuit,
    estimator=backend,
    gradient=gradient, 
    input_params=input_params,   # Assuming these are defined
    weight_params=weight_params
)
# TorchConnector handles the integration with PyTorch's autograd
model_g = TorchConnector(qnn_g)

# --- 4. Pre-compilation for KL Divergence Tracking ---
# Instead of instantiating Statevector() every loop, we pre-transpile
# a circuit that explicitly saves probabilities.
prob_circuit = generator_circuit.copy()
prob_circuit.save_probabilities() # GPU-native instruction [8]
transpiled_prob_circ = transpile(prob_circuit, backend)

# --- 5. Optimized Training Loop ---
for epoch in range(current_epoch, train_config['max_iterations']+1):

    # --- Quantum Discriminator Updates ---
    for disc_train_step in range(D_STEPS):
        # Optimization: Ensure closure_d handles batching correctly
        disc_loss = optimizer_d.step(closure_d)
        if (disc_train_step == D_STEPS-1):
            dloss[epoch] = disc_loss.detach().cpu().numpy()

    # --- Quantum Generator Updates ---
    for gen_train_step in range(G_STEPS):
        # The optimizer uses ReverseEstimatorGradient implicitly via model_g
        gen_loss = optimizer_g.step(closure_g)
        if (gen_train_step == G_STEPS-1):
            gloss[epoch] = gen_loss.detach().cpu().numpy()

    # --- Optimized KL Tracking ---
    # We avoid creating a new simulation instance. We run the pre-transpiled
    # circuit on the persistent GPU backend.
    with torch.no_grad():
        # Get current weights efficiently
        gen_params_tensor = torch.nn.utils.parameters_to_vector(model_g.parameters())
        gen_params_np = gen_params_tensor.cpu().numpy()
        
        # Execute on GPU backend; retrieve only small probability vector
        # parameter_binds maps the circuit parameters to the current numpy weights
        job = backend.run(
            transpiled_prob_circ, 
            parameter_binds=[{p: v for p, v in zip(prob_circuit.parameters, gen_params_np)}]
        )
        result = job.result()
        
        # Data transfer is minimal: 2^N floats instead of complex statevector
        probs_np = result.data(0)['probabilities'] 
        
        # Compute KL using PyTorch (can be done on GPU if tensors are moved there)
        gen_distribution_tensor = torch.from_numpy(probs_np)
        current_kl = torch.nn.functional.kl_div(
            input=gen_distribution_tensor.log(), 
            target=real_distribution_tensor, 
            reduction='sum'
        ).item()
        
        kl_div[epoch] = current_kl
        
        if min_kl_div > current_kl:
            min_kl_div = current_kl
            best_gen_params = gen_params_np

NameError: name 'gradient' is not defined

In [ ]:
value_dcost_fake = disc_fake_qnn.forward(gen_params.detach(), disc_params.detach())[0,0]
value_dcost_real = disc_real_qnn.forward([], disc_params.detach())[0,0]


grad_dcost_fake = disc_fake_qnn.backward(gen_params.detach(), disc_params.detach())[1][0,0]
grad_dcost_real = disc_real_qnn.backward([], disc_params.detach())[1][0,0]


print(value_dcost_fake, value_dcost_real, grad_dcost_fake, grad_dcost_real)

-0.07614865276264027 -0.24768847867380828 [ 1.88213447e-01  2.54346660e-02 -3.53355673e-01 -4.06557016e-01
 -6.80829957e-02  1.56869657e-01  9.21908102e-02 -2.92552537e-02
  1.04670972e-01  3.23729215e-01 -2.51169527e-01  9.26615763e-02
 -2.77555756e-17  6.07153217e-17 -5.55111512e-17  1.60215298e-01
  4.40572509e-01  2.77555756e-17] [ 1.01028582e-01 -1.82956686e-01 -4.54795341e-02  4.54795341e-02
 -1.42755927e-01  1.64926736e-01 -2.84142196e-01 -3.47703140e-01
 -6.73684881e-02  8.70879370e-02 -4.63491040e-01  3.38541303e-01
 -8.32667268e-17 -1.38777878e-17  8.32667268e-17  1.04554093e-01
 -1.10651408e-01  0.00000000e+00]


In [ ]:
prob_circuit = generator_circuit.copy()
prob_circuit.save_probabilities() # GPU-native instruction [8]
transpiled_prob_circ = transpile(prob_circuit, backend)